#**LangChain**

#**Step 01: Install All the Required Packages**

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 824.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.5 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

#**Step 03: Import All the Required Libraries**

In [ ]:
from langchain.llms import HuggingFacePipeline



AutoTokenizer. A tokenizer is responsible for preprocessing text into an array of numbers as inputs to a model.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
import transformers

In [ ]:
import torch

In [ ]:
import warnings
warnings.filterwarnings('ignore')


##**Step 04: Load the Llama 2 Model**

In [ ]:
model="meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model)

In [ ]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [ ]:
prompt="What would be a good name for a company that makes colorful socks"

In [ ]:
print(llm(prompt))

In [ ]:
prompt="I want to open a restaurant for  Chinese food. Suggest me a fence name for this"

In [ ]:
print(llm(prompt))

##**Step 05: Prompt Templates**

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
from langchain.chains import LLMChain

#**Example 1**

In [ ]:
prompt_template=PromptTemplate(input_variables=["cuisine"],
                               template="I want to open a restaurant for {cuisine} food. Suggest a fency name for this")

In [ ]:
input_prompt=prompt_template.format(cuisine="Italian")

In [ ]:
print(input_prompt)

#**Example 2**

In [ ]:
prompt_template=PromptTemplate(input_variables=["book_name"],
                               template="Privide me a concise summary of the book {book_name}")

In [ ]:
input_prompt=prompt_template.format(book_name="Alchemist")

In [ ]:
print(input_prompt)

#**Example 1**

In [ ]:
prompt_template=PromptTemplate(input_variables=["book_name"],
                               template="Proivide me a concise summary of the book {book_name}")

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)
response= chain.run("Alchemist")
print(response)

#**Example 2**

In [ ]:
prompt_template=PromptTemplate(input_variables=["cuisine"],
                               template="I want to open a restaurant for {cuisine} food. Suggest me a fency name for this")

In [ ]:
chain=LLMChain(llm=llm, prompt=prompt_template, verbose=True)

In [ ]:
response=chain.run("Italian")
print(response)

#Import the Required Library

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
prompt_template_one=PromptTemplate(input_variables=["cuisine"],
                               template="I want to open a restaurant for {cuisine} food. Suggest me a fency name for this")


In [ ]:
restaurant_name_chain=LLMChain(llm=llm, prompt=prompt_template_one)

In [ ]:
prompt_template_two=PromptTemplate(input_variables=["restaurant_name"],
                                   template="""You are provided with a lot of restaurant names: {restaurant_name}, just pick the first
                                   restaurant name and suggest some menu items for the restaurant""")

In [ ]:
restaurant_menu_items_chain=LLMChain(llm=llm, prompt=prompt_template_two)

In [ ]:
chain=SimpleSequentialChain(chains=[restaurant_name_chain, restaurant_menu_items_chain])
chain.run("Italian")


#Import the Required Library

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
prompt_template_one=PromptTemplate(input_variables=["cuisine"],
                                   template="""I want to open a restaurant for {cuisine} food. Suggest me a fency
                                   name for this, please only provide me one restaurant name""")

In [ ]:
restaurant_name_chain=LLMChain(llm=llm, prompt=prompt_template_one, output_key="restaurant_name")

In [ ]:
prompt_template_two=PromptTemplate(input_variables=["restaurant_name"],
                                   template="""Suggest some menu items for the restaurant {restaurant_name}""")

In [ ]:
restaurant_menu_items_chain=LLMChain(llm=llm, prompt=prompt_template_two, output_key="menu_items")

In [ ]:
chain=SequentialChain(chains=[restaurant_name_chain, restaurant_menu_items_chain],
                      input_variables=["cuisine"],
                      output_variables=["restaurant_name", "menu_items"])

In [ ]:
chain({"cuisine":"Italian"})

In [ ]:
chain({"cuisine":"Paistani"})

##**Step 07: Memory**

Chatbot application, you will notice that it remember past information

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables =['product'],
    template = "What is a good name for a company that makes {product}"
)

In [ ]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("colorful socks")
print(name)

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables =['product'],
    template = "What would be  a good name for a company that makes {product}"
)


In [ ]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)


In [ ]:
name = chain.run("Security Cameras")
print(name)

In [ ]:
chain.memory

In [ ]:
type(chain.memory)